# Day 17

In [31]:
from itertools import takewhile
def parse_range(rng):
    _, rng = rng.strip().split('=')
    return [int(n) for n in rng.split('..')]

def parse_target_area(s):
    _, s = s.split(':')
    return [parse_range(rng) for rng in s.strip().split(',')]

def probe(vx, vy):
    x, y = 0, 0
    while True:
        x += vx
        y += vy
        if vx != 0:
            vx -= 1 if vx > 0 else -1
        vy -= 1
        yield x, y
        
def shoot(vx, vy, xrng, yrng):
    g = probe(vx, vy)
    return [*takewhile(
        lambda t: t[0] <= xrng[1] and t[1] >= yrng[0],
        g
    )]
    
def find_length(arr):
    # print(arr)
    a, b = min(arr), max(arr)
    if b > 0:
        offset = b
        if a <= 0:
            length = b - a
        else:
            length = b
    else:
        length, offset = -a, 0
    return length + 1, offset

def find_dimensions(path, xrng, yrng):
    xarr = (x for x, _ in path)
    yarr = (y for _, y in path)
    xarr = [*xarr, *xrng]
    yarr = [*yarr, *yrng]
    return [find_length(arr) for arr in (xarr, yarr)]

def make_canvas(*, xlen, ylen):
    return [
        ['.'] * xlen
        for _ in range(ylen)
    ]

def chart_point(symbol, x, y, *, canvas, xofs, yofs):
    canvas[y + yofs][x + xofs] = symbol
    
def chart_target_area(xrng, yrng, *, canvas, xofs, yofs):
    for x in range(xrng[0], xrng[1]):
        for y in range(yrng[0], yrng[1]):
            try:
                chart_point('T', x, y, canvas=canvas, xofs=xofs, yofs=yofs)
            except:
                print(x, y)
    
def shoot_and_chart(vx, vy, xrng, yrng):
    path = shoot(vx, vy, xrng, yrng)
    print(path)
    (xlen, xofs), (ylen, yofs) = find_dimensions(path, xrng, yrng)
    print((xlen, xofs), (ylen, yofs))
    canvas = make_canvas(xlen=xlen, ylen=ylen)
    chart_point('S', 0, 0, canvas=canvas, xofs=xofs, yofs=yofs)
    chart_target_area(xrng, yrng, canvas=canvas, xofs=xofs, yofs=yofs)
    return canvas

def show(canvas):
    for row in canvas:
        print(' '.join(row))
        
s = 'target area: x=20..30, y=-10..-5'
xrng, yrng = parse_target_area(s)
canvas = shoot_and_chart(6, 3, xrng, yrng)
show(canvas)

[(6, 3), (11, 5), (15, 6), (18, 6), (20, 5), (21, 3), (21, 0), (21, -4), (21, -9)]
(31, 30) (17, 6)
20 -10
20 -9
20 -8
20 -7
20 -6
21 -10
21 -9
21 -8
21 -7
21 -6
22 -10
22 -9
22 -8
22 -7
22 -6
23 -10
23 -9
23 -8
23 -7
23 -6
24 -10
24 -9
24 -8
24 -7
24 -6
25 -10
25 -9
25 -8
25 -7
25 -6
26 -10
26 -9
26 -8
26 -7
26 -6
27 -10
27 -9
27 -8
27 -7
27 -6
28 -10
28 -9
28 -8
28 -7
28 -6
29 -10
29 -9
29 -8
29 -7
29 -6
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . S
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . 